In [13]:
from keras.models import Sequentialuential

In [14]:
model=Sequential()
model.add(Embedding(voc, hidden_size, input_length))

W0816 09:31:27.921162  9792 deprecation_wrapper.py:119] From C:\Users\user\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.



In [ ]:
#model.add(Dense(2,inputt_dim=4, init='uniform', activation='linear'))
model=Sequential()
model.add(Dense(4, input_dim=10, init='normal', acitviation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.summary():요약정보
model.compile():함수설정
model.fit():학습
model.evalute():모델 평가
model.predict():예측
model.save(), load_model(): 모델 저장&불러오기

In [17]:
import tensorflow as tf
tf.__version__
tf.get_default_graph() #기본 그래프 정보

In [34]:
testGraph=tf.Graph() #사용자 그래프 생성
with testGraph.as_default(): #사용자 그래프에 노드 정의
    x=tf.constant(10)
    y=x+20

In [30]:
# Session():세션객체생성, 분산처리환경 노드를 연결
# run():연산 수행 -> 결과를 리턴
# close():세션 종료
x=tf.constant(5)
y=x+10
sess=tf.Session()
print(sess.run(x))
print(sess.run(y))
sess.close()

5
15


In [31]:
with tf.Session() as sess: #특별히 그래프를 지정하지 않았으므로, 기본그래프를 사용해서 세션을 실행
    print(sess.run(x))

5


In [35]:
with tf.Session(graph=testGraph) as sess: #특별히 그래프를 지정하지 않았으므로, 기본그래프를 사용해서 세션을 실행
    print(sess.run(x))

10


In [38]:
# sess=tf.InteractiveSession()
# x.eval()
# y.eval()

In [39]:
var1=tf.Variable(5, name="var1")
var2=tf.add(var1, 5, name="var2")
var1.op.name

'var1'

In [41]:
#이름공간(name space,폴더와 비슷)을 이용한 노드 이름 중복 방지
#tf.name_scope():이름 공간 생성
with tf.name_scope("ns1"):
    v3=tf.Variable(1, name="var3")
    v4=tf.add(v3, 2,name="var4")
print(v3.op.name)

ns1_1/var3


In [139]:
#입력 4개 -> 출력 2개(hidden_size)

#"hihello"
# "hihell" => 입력
# "ihello" => 출력
#time_step:6, input_dim:5, hidden_size:5

idx2char=['h', 'i', 'e', 'l', 'o']
xdata=[[0,1,0,2,3,3]] # xdata="hihell"
                     #        "ihello"
        #   cell -> ihello
x_onehot=[[
     [1,0,0,0,0],
     [0,1,0,0,0],
     [1,0,0,0,0],
     [0,0,1,0,0],
     [0,0,0,1,0],
     [0,0,0,1,0]]]
ydata=[[1,0,2,3,3,4]] #"ihello"

tf.reset_default_graph()

num_classes=5
input_dim=5 #입력데이터의 차원(단어벡터의 길이)
hidden_size=5
batch_size=1
sequence_length=6 #hihell
learning_rate=0.1

x=tf.placeholder(tf.float32, [None,sequence_length,input_dim])
y=tf.placeholder(tf.int32, [None,sequence_length])

cell=tf.contrib.rnn.BasicLSTMCell(num_units=hidden_size)
ini_state=cell.zero_state(batch_size, tf.float32)
outputs, _states=tf.nn.dynamic_rnn(cell, x, initial_state=ini_state, dtype=tf.float32)

x_fc=tf.reshape(outputs, [-1, hidden_size])
outputs=tf.contrib.layers.fully_connected(inputs=x_fc, num_outputs=num_classes)
outputs=tf.reshape(outputs, [batch_size,sequence_length,num_classes])
#                           [1, 6, 5] =>[[[0.1,0.2,0.5,0.1,0.1] ],[ ],...,[ ]]

weights=tf.ones([batch_size, sequence_length])
sequence_loss=tf.contrib.seq2seq.sequence_loss(logits=outputs,targets=y, weights=weights)
loss=tf.reduce_mean(sequence_loss)

prediction=  tf.argmax(outputs,axis=2)

train=tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(101):
        lv, _=sess.run([loss, train], 
                       feed_dict={x:x_onehot, y:ydata})
        res=sess.run(prediction,feed_dict={x:x_onehot})
        print("step:",i,"cost:",lv, "prediction:", res, 
              "true Y:", ydata)
        resStr=[idx2char[c] for c in np.squeeze(res)]
        print("predction str:","".join(resStr))
        # prediction:[[1 0 2 2 3 1]]  true Y: [[1 0 2 3 3 4]]
        #....
        # prediction:[[1 0 2 3 3 4]]  true Y: [[1 0 2 3 3 4]]
        
        


#num_units는 출력개수




#학습 hihell -> ihello

# hihell -> elhill
# ...
# hihell -> ihehho
# ...
# hihell -> ihello


step: 0 cost: 1.6100149 prediction: [[3 3 3 3 3 3]] true Y: [[1, 0, 2, 3, 3, 4]]
predction str: llllll
step: 1 cost: 1.5471888 prediction: [[3 3 3 3 3 3]] true Y: [[1, 0, 2, 3, 3, 4]]
predction str: llllll
step: 2 cost: 1.4851211 prediction: [[3 3 3 3 3 3]] true Y: [[1, 0, 2, 3, 3, 4]]
predction str: llllll
step: 3 cost: 1.4394479 prediction: [[3 3 3 3 3 3]] true Y: [[1, 0, 2, 3, 3, 4]]
predction str: llllll
step: 4 cost: 1.3958031 prediction: [[3 3 3 3 3 3]] true Y: [[1, 0, 2, 3, 3, 4]]
predction str: llllll
step: 5 cost: 1.3304477 prediction: [[3 3 3 3 3 3]] true Y: [[1, 0, 2, 3, 3, 4]]
predction str: llllll
step: 6 cost: 1.2376846 prediction: [[3 0 3 3 3 3]] true Y: [[1, 0, 2, 3, 3, 4]]
predction str: lhllll
step: 7 cost: 1.1362847 prediction: [[2 0 2 3 3 3]] true Y: [[1, 0, 2, 3, 3, 4]]
predction str: ehelll
step: 8 cost: 1.0371956 prediction: [[2 0 2 3 3 4]] true Y: [[1, 0, 2, 3, 3, 4]]
predction str: ehello
step: 9 cost: 0.93446547 prediction: [[2 0 2 3 4 4]] true Y: [[1, 0, 2, 3

step: 90 cost: 0.0020860608 prediction: [[1 0 2 3 3 4]] true Y: [[1, 0, 2, 3, 3, 4]]
predction str: ihello
step: 91 cost: 0.0020560394 prediction: [[1 0 2 3 3 4]] true Y: [[1, 0, 2, 3, 3, 4]]
predction str: ihello
step: 92 cost: 0.0020270066 prediction: [[1 0 2 3 3 4]] true Y: [[1, 0, 2, 3, 3, 4]]
predction str: ihello
step: 93 cost: 0.0019987447 prediction: [[1 0 2 3 3 4]] true Y: [[1, 0, 2, 3, 3, 4]]
predction str: ihello
step: 94 cost: 0.001971333 prediction: [[1 0 2 3 3 4]] true Y: [[1, 0, 2, 3, 3, 4]]
predction str: ihello
step: 95 cost: 0.0019447127 prediction: [[1 0 2 3 3 4]] true Y: [[1, 0, 2, 3, 3, 4]]
predction str: ihello
step: 96 cost: 0.001918884 prediction: [[1 0 2 3 3 4]] true Y: [[1, 0, 2, 3, 3, 4]]
predction str: ihello
step: 97 cost: 0.0018937476 prediction: [[1 0 2 3 3 4]] true Y: [[1, 0, 2, 3, 3, 4]]
predction str: ihello
step: 98 cost: 0.0018693238 prediction: [[1 0 2 3 3 4]] true Y: [[1, 0, 2, 3, 3, 4]]
predction str: ihello
step: 99 cost: 0.0018455532 prediction:

In [ ]:
#"honggildong"

In [134]:
test=[[3,1,0,2,1]]
#np.squeeze(test)
test2=[idx2char[c] for c in np.squeeze(test)]
test2

['l', 'i', 'h', 'e', 'i']

In [107]:
import numpy as np
from tensorflow.contrib import rnn
import pprint

tf.reset_default_graph()
pp=pprint.PrettyPrinter(indent=4)
sess=tf.InteractiveSession()


In [108]:
h=[1,0,0,0] #hello => 4글자
e=[0,1,0,0]
l=[0,0,1,0]
o=[0,0,0,1]

In [97]:
with tf.variable_scope('first_cell') as scope:
    hidden_size=2
    cell=tf.contrib.rnn.BasicRNNCell(num_units=hidden_size)
    print(cell.output_size, cell.state_size)
    xdata=np.array([[h]], dtype=np.float32)
    outputs, _states=tf.nn.dynamic_rnn(cell,xdata, dtype=tf.float32)
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())
    pp.pprint(_states.eval())
    
    

2 2
array([[[-0.59038234, -0.32656184]]], dtype=float32)
array([[-0.59038234, -0.32656184]], dtype=float32)


In [104]:
with tf.variable_scope('sec_cell') as scope:
    hidden_size=6
    cell=tf.contrib.rnn.BasicRNNCell(num_units=hidden_size)
    xdata=np.array([[h,e,l,l,o]], dtype=np.float32)
    print(xdata.shape)
#(1,5,4)
#(batch_size,seq_len,input_dim)
    pp.pprint(xdata)
    outputs, _states=tf.nn.dynamic_rnn(cell,xdata, dtype=tf.float32)
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())   
    

(1, 5, 4)
array([[[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]]], dtype=float32)
array([[[ 0.39774477,  0.5468991 ],
        [ 0.8928871 , -0.04237348],
        [-0.01551964, -0.12247499],
        [-0.57187355, -0.40627557],
        [-0.80547315, -0.36207783]]], dtype=float32)


In [109]:
with tf.variable_scope('thr_cell') as scope:    
    hidden_size=2
    cell=tf.contrib.rnn.BasicRNNCell(num_units=hidden_size)
    xdata=np.array([[h,e,l,l,o],
                   [e,o,l,l,l],
                   [l,l,e,e,l]], dtype=np.float32)
    pp.pprint(xdata)    
    outputs, _states=tf.nn.dynamic_rnn(cell,xdata, dtype=tf.float32)
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())   
    

array([[[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]],

       [[0., 1., 0., 0.],
        [0., 0., 0., 1.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.]],

       [[0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.]]], dtype=float32)
array([[[ 0.5933645 ,  0.33998343],
        [-0.3217429 ,  0.6085877 ],
        [ 0.32240725,  0.33100843],
        [ 0.5205061 ,  0.32865316],
        [ 0.5103742 ,  0.41573536]],

       [[-0.15534404,  0.6055428 ],
        [ 0.30791327,  0.41788206],
        [ 0.47565743,  0.32958195],
        [ 0.53085035,  0.32879436],
        [ 0.5346692 ,  0.32884225]],

       [[ 0.64783275,  0.32472104],
        [ 0.54383093,  0.32891762],
        [-0.3189336 ,  0.60846704],
        [-0.5324154 ,  0.6099864 ],
        [ 0.30503196,  0.33080855]]], dtype=float32)


In [110]:
#M(시트),V(그래프),C(컨트롤러(막대)) 분리

with tf.variable_scope('lstm_thr_cell') as scope:    
    hidden_size=2
    cell=tf.contrib.rnn.BasicLSTMCell(num_units=hidden_size)
    xdata=np.array([[h,e,l,l,o],
                   [e,o,l,l,l],
                   [l,l,e,e,l]], dtype=np.float32)
    pp.pprint(xdata)    
    outputs, _states=tf.nn.dynamic_rnn(cell,xdata, dtype=tf.float32)
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())   
    

W0816 14:47:28.700894  9792 deprecation.py:323] From <ipython-input-110-2a466152e999>:5: BasicLSTMCell.__init__ (from tensorflow.python.ops.rnn_cell_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.


array([[[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]],

       [[0., 1., 0., 0.],
        [0., 0., 0., 1.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.]],

       [[0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.]]], dtype=float32)
array([[[-0.1322009 ,  0.04252621],
        [-0.16804105, -0.04627199],
        [-0.01589115, -0.12874529],
        [ 0.15109849, -0.18411236],
        [ 0.10181877, -0.20873772]],

       [[-0.09788718, -0.06387524],
        [-0.02616269, -0.14750491],
        [ 0.13693146, -0.21632285],
        [ 0.26180714, -0.24035087],
        [ 0.34520617, -0.2489305 ]],

       [[ 0.1574434 , -0.0753042 ],
        [ 0.26998237, -0.12255721],
        [ 0.09413569, -0.15500641],
        [-0.02562424, -0.18844967],
        [ 0.14212008, -0.22042999]]], dtype=float32)


In [ ]:
with tf.variable_scope('lstm_thr_cell_2') as scope:    
    hidden_size=2
    cell=tf.contrib.rnn.BasicLSTMCell(num_units=hidden_size)
    xdata=np.array([[h,e,l,l,o],
                   [e,o,l,l,l],
                   [l,l,e,e,l]], dtype=np.float32)
    pp.pprint(xdata)    
    outputs, _states=tf.nn.dynamic_rnn(cell,xdata,sequence_length=[5,3,4], dtype=tf.float32)
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())   
    

In [ ]:

with tf.variable_scope('init_lstm_thr_cell') as scope:    
    hidden_size=2
    batch_size=3
    cell=tf.contrib.rnn.BasicLSTMCell(num_units=hidden_size)
    xdata=np.array([[h,e,l,l,o],
                   [e,o,l,l,l],
                   [l,l,e,e,l]], dtype=np.float32)
    pp.pprint(xdata)    
    init_state=cell.zero_state(batch_size, tf.float32)
    outputs, _states=tf.nn.dynamic_rnn(cell,xdata,initial_state=init_state ,dtype=tf.float32)
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())   
    
    
#     데이터:100개
#         1에폭:100개 데이터를 학습
#     batch_size:1개, batch_length:100/20=5번
#             (20개 단위로 학습(forward,backward))
        
    
    

In [117]:
batch_size=3
seq_len=5
in_dim=3
xdata=np.arange(45,dtype=np.float32).reshape(batch_size,seq_len,in_dim)
print(xdata)

[[[ 0.  1.  2.]
  [ 3.  4.  5.]
  [ 6.  7.  8.]
  [ 9. 10. 11.]
  [12. 13. 14.]]

 [[15. 16. 17.]
  [18. 19. 20.]
  [21. 22. 23.]
  [24. 25. 26.]
  [27. 28. 29.]]

 [[30. 31. 32.]
  [33. 34. 35.]
  [36. 37. 38.]
  [39. 40. 41.]
  [42. 43. 44.]]]


In [118]:

with tf.variable_scope('multi_rnn_cell') as scope:    

    cell=tf.contrib.rnn.BasicLSTMCell(num_units=5)
    cell=tf.contrib.rnn.MultiRNNCell([cell]*3)#layer 3개
    outputs, _states=tf.nn.dynamic_rnn(cell, xdata,dtype=tf.float32)
    sess.run(tf.global_variables_initializer())
    pp.print(outputs.eval())  
    
    

ValueError: in converted code:
    relative to C:\Users\user\Anaconda3\lib\site-packages\tensorflow\python:

    ops\rnn_cell_impl.py:1719 call *
        cur_inp, new_state = cell(cur_inp, cur_state)
    ops\rnn_cell_impl.py:385 __call__
        self, inputs, state, scope=scope, *args, **kwargs)
    ops\rnn_cell_impl.py:766 call *
        gate_inputs = math_ops.matmul(
    util\dispatch.py:180 wrapper
        return target(*args, **kwargs)
    ops\math_ops.py:2647 matmul
        a, b, transpose_a=transpose_a, transpose_b=transpose_b, name=name)
    ops\gen_math_ops.py:6294 mat_mul
        name=name)
    framework\op_def_library.py:788 _apply_op_helper
        op_def=op_def)
    util\deprecation.py:507 new_func
        return func(*args, **kwargs)
    framework\ops.py:3616 create_op
        op_def=op_def)
    framework\ops.py:2027 __init__
        control_input_ops)
    framework\ops.py:1867 _create_c_op
        raise ValueError(str(e))

    ValueError: Dimensions must be equal, but are

In [ ]:
ydata=tf.constant([[1,1,1]])
prediction1=tf.constant([[[0.3, 0.7], [0.1, 0.9], [0.3, 0.7]])
prediction2=tf.constant([[[0.4, 0.6], [0.4, 0.6], [0.6, 0.4]])                         
                         
loss1=tf.contrib.seq2seq.sequence_loss(prediction1, ydata)
sess.run(tf.global_variables_initializer())
loss1.eval()                         
                                     